# Open Source Object-Based-Image-Analysis
*Example workflow to classify an urban scene into basic classes: trees, water, open space, buildings, pavement, shadow*

*The great thing about python is...*  
<img src="img/build_it2.png" width="600">

### What is OBIA really?
Derivatives -> Segmentation -> Zonal statistics -> Classification 

## Tools / Software Stack

## python
# ![python](img/python_logo.png)  +  ![gdal](img/gdalicon.png)  
- [geopandas](https://geopandas.org): vector (image object) manipulation  
- [rasterio](https://rasterio.readthedocs.io/en/latest/): raster loading/manipulation  
- [rasterstats](https://pythonhosted.org/rasterstats/): zonal statistics  
*The use of anaconda/miniconda is *highly* recommended for managing python dependencies, specifically installing packages from the `conda-forge` channel. E.g.:*  
`conda install -c conda-forge geopandas`
  
  
## WhiteBoxTools
![wbt](img/WhiteBoxToolsLogo_vert.png)  
[WhiteBoxTools](https://jblindsay.github.io/wbt_book/intro.html): raster derivatives, specifically geomorphometic analyses.  
*Has python bindings, but tough to integrate with other dependencies, so called from the command line.*  
  
  
## Orfeo Toolbox  
![orfeo](img/logo-orfeo-toolbox.png)  
[**Orfeo Toolbox**](https://www.orfeo-toolbox.org/CookBook/): segmentation (also has general GIS tools).  
*Has python bindings, but tough to integrate with other dependencies, so called from the command line.*  


## QGIS
![qgis](img/qgis-logo.png)  
[**QGIS**](https://qgis.org/en/site/): Viewing outputs, general GIS.

## Imports + Logging Set up

In [1]:
# Standard lib
import copy
import logging
import numpy as np
import operator
from pathlib import Path
from pprint import pprint
import warnings
import sys

# Installed packages
from osgeo import gdal, osr
import geopandas
import rasterio
import rasterstats
from skimage.segmentation import quickshift
import scipy

# Local packages
from lib import (run_subprocess, clean4cmdline, create_grm_outname,
                 rio_polygonize, read_vec, write_gdf, write_array,)
from calc_zonal_stats import calc_zonal_stats

In [2]:
# Set up logger
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(1)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root_logger.addHandler(ch)

warnings.filterwarnings('ignore', category=RuntimeWarning, message='Sequential read of iterator*')

## Paths (Orfeo + Data)

In [3]:
# Orfeo setup and tools
# Change this to match where OTB is on your machine, the rest of the paths should be OK
otb = Path(r'C:\OTB-7.2.0-Win64\OTB-7.2.0-Win64')
otb_init = otb / 'otbenv.bat'
otb_bin = otb / 'bin'
otb_grm = otb_bin / 'otbcli_GenericRegionMerging.bat'
# otb_lsms = otb_bin / 'otbcli_LargeScaleMeanShift.bat'
otb_grm

WindowsPath('C:/OTB-7.2.0-Win64/OTB-7.2.0-Win64/bin/otbcli_GenericRegionMerging.bat')

In [4]:
# Data
data_dir = Path(r'./data')
img = data_dir / 'naip_m_4509361_se_15_060_20190727_aoi.tif'
ndsm = data_dir / 'nDSM_clip_fill.tif'
ndvi = data_dir / 'ndvi_naip_m_4509361_se_15_060_20190727_aoi.tif'
roughness = data_dir / 'nDSM_clip_fill_roughness.tif'
img

WindowsPath('data/naip_m_4509361_se_15_060_20190727_aoi.tif')

In [5]:
# Out paths
out_dir = Path(r'./results')
seg_dir = out_dir / 'seg'

In [6]:
# Ensure all exist
missing_files = []
for file_path in [data_dir, img, ndsm, ndvi, roughness, out_dir]:
    if not file_path.exists():
        missing_files.append(file_path)
if len(missing_files) > 0:
    for file_path in missing_files:
        root_logger.error(f'Missing file/folder: {file_path}')
else:
    root_logger.info('All files/folders located.')

2021-04-23 11:23:31,932 - root - INFO - All files/folders located.


### Check out input data
how it was created, histogram of values in image (qgis), profile, stats of image to seg

In [7]:
response = run_subprocess(f'gdalinfo {img} -stats', log=False)
for l in response:
    print(l.strip('\n'))

Driver: GTiff/GeoTIFF
Files: data\naip_m_4509361_se_15_060_20190727_aoi.tif
       data\naip_m_4509361_se_15_060_20190727_aoi.tif.aux.xml
Size is 2506, 1444
Coordinate System is:
PROJCRS["NAD83 / UTM zone 15N",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["UTM zone 15N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-93,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False e

## Segmentation

#### Generic Region Merging
[otb docs](https://www.orfeo-toolbox.org/CookBook/Applications/app_GenericRegionMerging.html?highlight=generic%20region%20merging)

In [8]:
# Parameters
# Homogeneity criterion to use. The default is 'bs'. One of: [bs, ed, fls]
criterion = 'bs'
threshold = 100
niter = 100
spectral_w = 0.6 # spectral weight, higher values slow processing time
spatial_w = 25 # spatial weight
out_img = create_grm_outname(img=img,
                             out_dir=seg_dir,
                             criterion=criterion,
                             threshold=threshold,
                             niter=niter,
                             spectral=spectral_w,
                             spatial=spatial_w)
root_logger.info(f'Out image: {out_img}')

2021-04-23 11:23:32,440 - root - INFO - Out image: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.tif


In [9]:
# Build the command
otb_cmd = f"""
"{otb_grm}"
-in {str(img)}
-out {str(out_img)}
-criterion {criterion}
-threshold {threshold}
-niter {niter}
-cw {spectral_w}
-sw {spatial_w}"""

otb_cmd = clean4cmdline(otb_cmd)
root_logger.info(f'OTB command:\n{otb_cmd}')
otb_cmd = f'{otb_init} && {otb_cmd}'
root_logger.debug(f'OTB full command:\n{otb_cmd}')

2021-04-23 11:23:32,474 - root - INFO - OTB command:
"C:\OTB-7.2.0-Win64\OTB-7.2.0-Win64\bin\otbcli_GenericRegionMerging.bat" -in data\naip_m_4509361_se_15_060_20190727_aoi.tif -out results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.tif -criterion bs -threshold 100 -niter 100 -cw 0.6 -sw 25


In [10]:
run_subprocess(otb_cmd)

2021-04-23 11:23:43,427 - lib - INFO - 2021-04-23 11:23:32 (INFO) GenericRegionMerging: Default RAM limit for OTB is 256 MB

2021-04-23 11:23:43,431 - lib - INFO - 2021-04-23 11:23:32 (INFO) GenericRegionMerging: GDAL maximum cache size is 400 MB

2021-04-23 11:23:43,434 - lib - INFO - 2021-04-23 11:23:32 (INFO) GenericRegionMerging: OTB will use at most 8 threads

2021-04-23 11:24:34,955 - lib - INFO - ....................................................................................................

2021-04-23 11:24:37,861 - lib - INFO - 2021-04-23 11:24:37 (INFO): Estimated memory for full processing: 68.9824MB (avail.: 256 MB), optimal image partitioning: 1 blocks

2021-04-23 11:24:37,862 - lib - INFO - 2021-04-23 11:24:37 (INFO): File results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.tif will be written in 1 blocks of 2506x1444 pixels

2021-04-23 11:24:37,863 - lib - INFO - Writing results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6s

[]

(Check out resulting image)

In [21]:
# Convert output tif to polygons
out_vec = out_img.replace('.tif', '.gpkg/seg')
vec_objects = rio_polygonize(img=out_img, out_vec=out_vec)

2021-04-23 11:32:31,729 - lib - INFO - Polygonizing: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.tif
2021-04-23 11:32:32,546 - lib - INFO - Writing polygons to: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.gpkg/seg


(Check out resuting polygons)

### Other segmentation options

[Orfeo Toolbox LargeScaleMeanShift](https://www.orfeo-toolbox.org/CookBook/Applications/app_LargeScaleMeanShift.html)

[Scikit-Image Segmenation](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_segmentations.html)  
Inputs and output are often numpy arrays/matrices that need to be converted from geographic-space to pixel-space and back to geographic-space. 

![pixel_space](img/pixel_space.png)

A couple of useful functions for this conversion:

In [13]:
# Functions for going back and forth from pixel space to coordinate space
def pixel2geo(pixel_coord, geotransform):
    """
    Covert pixel coordinates to geographic coordinates
    """
    y, x = pixel_coord
    gy = geotransform[4] * x + geotransform[5] * y + geotransform[4] * 0.5 + geotransform[5] * 0.5 + geotransform[3]
    gx = geotransform[1] * x + geotransform[2] * y + geotransform[1] * 0.5 + geotransform[2] * 0.5 + geotransform[0]

    return gy, gx

def geo2pixel(geocoord, geotransform):
    """
    Convert geographic coordinates to pixel coordinates
    """
    y, x = geocoord
    py = int(np.around((y - geotransform[3]) / geotransform[5]))
    px = int(np.around((x - geotransform[0]) / geotransform[1]))
    return py, px

Luckily GDAL can handle much of this work, if you know where to look...  
<img src="img/gdal_geotransform.png" width="400">


In [11]:
# Open image and get geotransform
ds = gdal.Open(str(img))
geotransform = ds.GetGeoTransform()

# Log info 
root_logger.info(f'Geotransform:\n{geotransform}\n')
root_logger.info(f'Top left x: {geotransform[0]}')
root_logger.info(f'X Resolution: {geotransform[1]}')
root_logger.info(f'Rotation1: {geotransform[2]}')
root_logger.info(f'Top left y: {geotransform[3]}')
root_logger.info(f'Rotation2: {geotransform[4]}')
root_logger.info(f'Y Resolution: {geotransform[5]}\n')

2021-04-23 11:24:53,337 - root - INFO - Geotransform:
(467491.8, 0.600000000000014, 0.0, 4983821.4, 0.0, -0.600000000000258)

2021-04-23 11:24:53,340 - root - INFO - Top left x: 467491.8
2021-04-23 11:24:53,342 - root - INFO - X Resolution: 0.600000000000014
2021-04-23 11:24:53,345 - root - INFO - Rotation1: 0.0
2021-04-23 11:24:53,346 - root - INFO - Top left y: 4983821.4
2021-04-23 11:24:53,348 - root - INFO - Rotation2: 0.0
2021-04-23 11:24:53,350 - root - INFO - Y Resolution: -0.600000000000258



In [14]:
root_logger.info(f'Top left corner in pixels (y, x): {geo2pixel(geocoord=(4983821.4, 467491.8), geotransform=geotransform)}')
root_logger.info(f'Top left corner in geocoords (y, x): {pixel2geo(pixel_coord=(0,0), geotransform=geotransform)}')

2021-04-23 11:25:39,994 - root - INFO - Top left corner in pixels (y, x): (0, 0)
2021-04-23 11:25:39,996 - root - INFO - Top left corner in geocoords (y, x): (4983821.100000001, 467492.1)


#### Quickshift Segmentation

In [15]:
# Read image as array
array = ds.ReadAsArray()
# Reshape so that bands are last dimension of array
rgb = array.reshape(array.shape[1], array.shape[2], array.shape[0])
# Drop last band to get three-bands
rgb = rgb[:, :, :3]
# Run segmentation
labelled = quickshift(image=rgb, ratio=1, kernel_size=5, max_dist=25, sigma=1)

In [28]:
root_logger.info(f'Source array shape: {array.shape}')
root_logger.info(f'RGB array shape: {rgb.shape}')
root_logger.info(f'Labelled shape: {labelled.shape}')

2021-04-23 12:34:29,354 - root - INFO - Source array shape: (4, 1444, 2506)
2021-04-23 12:34:29,370 - root - INFO - RGB array shape: (1444, 2506, 3)
2021-04-23 12:34:29,370 - root - INFO - Labelled shape: (1444, 2506)


In [17]:
# Write array out, using geotransform+projection of original image
out_qs = out_img.replace('.tif', '_qs.tif')
write_array(labelled, out_qs, ds, dtype=3)

2021-04-23 11:30:02,364 - lib - WARNING - Unable to determine NoData value using -9999
2021-04-23 11:30:02,367 - lib - INFO - Creating raster at: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25_qs.tif
2021-04-23 11:30:02,542 - lib - INFO - Writing complete.


In [18]:
out_qs_vec = out_qs.replace('.tif', '_qs.gpkg/qs_seg')
qs_poly = rio_polygonize(img=out_qs, out_vec=out_qs_vec)

2021-04-23 11:30:03,654 - lib - INFO - Polygonizing: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25_qs.tif
2021-04-23 11:30:05,240 - lib - INFO - Writing polygons to: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25_qs_qs.gpkg/qs_seg


## Zonal Statistics

The [rasterstats](https://pythonhosted.org/rasterstats/https://pythonhosted.org/rasterstats/) package does all of the work here. The simpliest usage of `rasterstats` is very easy, this is straight from their docs:  
```python
from rasterstats import zonal_stats
zonal_stats("polygons.shp", "elevation.tif",
            stats="count min mean max median")
```
The only wonky bit is that this returns a `list` of `dicts`, one for each feature in `polygons.shp`:
```python
[...,
 {'count': 89,
  'max': 69.52958679199219,
  'mean': 20.08093536034059,
  'median': 19.33736801147461,
  'min': 1.5106816291809082},
]

```
The `calc_zonal_stats` function I wrote handles the logic of inputting multiple rasters, computing different stats for each raster, formatting the results into a GeoDataFrame and writing the output. It also adds the ability to compute `compactness` and `roundness`. 

In [19]:
# Create dictionary of rasters, stats, and bands to compute
zonal_stats_params = (
    {"img": {
        "path": img,
        "stats": ["mean", "max"],
        "bands": [1, 2, 3, 4]
	},
    "nDSM": {
        "path": ndsm,
        "stats": ["mean"]
    },
    "roughness":{
        "path": roughness,
        "stats": ["mean"],
    },
    "ndvi":{
        "path": ndvi,
        "stats": ["mean"]
    } 
})

In [29]:
out_zs = out_vec.replace('/seg', '/zs')
calc_zonal_stats(shp=out_vec,
                 rasters=[zonal_stats_params],
                 compactness=True,
                 roundness=True,
                 out_path=out_zs)

2021-04-23 12:35:36,574 - calc_zonal_stats - INFO - Reading in segments from: results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.gpkg/seg...
2021-04-23 12:35:37,361 - calc_zonal_stats - INFO - Segments found: 9,033
2021-04-23 12:35:37,361 - calc_zonal_stats - INFO - Computing mean max on raster:
	data\naip_m_4509361_se_15_060_20190727_aoi.tif
2021-04-23 12:35:37,361 - calc_zonal_stats - INFO - Band: 1


C:\Miniconda3\envs\osobia\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


2021-04-23 12:36:25,881 - calc_zonal_stats - INFO - Computing mean max on raster:
	data\naip_m_4509361_se_15_060_20190727_aoi.tif
2021-04-23 12:36:25,883 - calc_zonal_stats - INFO - Band: 2
2021-04-23 12:37:14,014 - calc_zonal_stats - INFO - Computing mean max on raster:
	data\naip_m_4509361_se_15_060_20190727_aoi.tif
2021-04-23 12:37:14,015 - calc_zonal_stats - INFO - Band: 3
2021-04-23 12:38:02,766 - calc_zonal_stats - INFO - Computing mean max on raster:
	data\naip_m_4509361_se_15_060_20190727_aoi.tif
2021-04-23 12:38:02,771 - calc_zonal_stats - INFO - Band: 4
2021-04-23 12:38:50,945 - calc_zonal_stats - INFO - Computing mean on raster:
	data\nDSM_clip_fill.tif
2021-04-23 12:39:29,329 - calc_zonal_stats - INFO - Computing mean on raster:
	data\nDSM_clip_fill_roughness.tif
2021-04-23 12:40:45,435 - calc_zonal_stats - INFO - Computing mean on raster:
	data\ndvi_naip_m_4509361_se_15_060_20190727_aoi.tif
2021-04-23 12:41:53,175 - calc_zonal_stats - INFO - Writing segments with statistic

,raster_val,geometry,imgb1_max,imgb1_mean,imgb2_max,imgb2_mean,imgb3_max,imgb3_mean,imgb4_max,imgb4_mean,nDSM_mean,roughness_mean,ndvi_mean,area_zs,compactness,roundness
0,1.0,"POLYGON ((467491.800 4983821.400, 467491.800 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,23.04,0.047589,21.013426
1,2.0,"POLYGON ((467530.200 4983821.400, 467530.200 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,23.04,0.047589,21.013426
2,3.0,"POLYGON ((467568.600 4983821.400, 467568.600 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,11.52,0.092315,10.832483
3,4.0,"POLYGON ((467587.800 4983821.400, 467587.800 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,5.76,0.173929,5.749472
4,5.0,"POLYGON ((467597.400 4983821.400, 467597.400 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,2.88,0.310281,3.222888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9028,8930.0,"POLYGON ((468683.400 4982980.800, 468683.400 4...",141.0,80.877061,155.0,93.407796,113.0,75.930285,201.0,129.033733,5.861141,11.850944,0.247328,480.24,0.251841,3.970761
9029,9032.0,"POLYGON ((468994.800 4982956.200, 468994.800 4...",0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,0.36,0.785398,1.273240
9030,8998.0,"POLYGON ((468471.600 4982967.600, 468471.600 4...",143.0,67.196319,131.0,67.527607,118.0,76.500000,131.0,31.723926,1.535610,2.468661,-0.414224,117.36,0.426555,2.344362
9031,9012.0,"POLYGON ((468607.200 4982964.600, 468607.200 4...",186.0,124.243802,176.0,123.235537,158.0,103.380165,142.0,108.235537,0.080337,0.392763,-0.012404,87.12,0.329976,3.030521


## Classification

This is done with python+geopandas but could also be done via QGIS+select_by_attribute

In [30]:
obj = read_vec(out_zs)
obj.sample(5)

,raster_val,imgb1_max,imgb1_mean,imgb2_max,imgb2_mean,imgb3_max,imgb3_mean,imgb4_max,imgb4_mean,nDSM_mean,roughness_mean,ndvi_mean,area_zs,compactness,roundness,geometry
2030,2031.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,0.36,0.785398,1.273240,"POLYGON ((468823.800 4983821.400, 468823.800 4..."
2244,2245.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,0.36,0.785398,1.273240,"POLYGON ((468952.200 4983821.400, 468952.200 4..."
8289,8294.0,110.0,58.528169,120.0,62.507042,118.0,69.672535,161.0,32.309859,0.558447,1.404757,-0.373246,102.24,0.305971,3.268280,"POLYGON ((468267.000 4983079.200, 468267.000 4..."
5221,5071.0,126.0,84.096257,145.0,96.530749,105.0,79.105615,202.0,149.094251,11.482721,14.327224,0.305672,538.56,0.381448,2.621588,"POLYGON ((468769.200 4983538.800, 468769.200 4..."
2409,2452.0,96.0,69.317073,99.0,67.975610,86.0,74.268293,93.0,33.902439,-0.005294,0.043333,-0.408481,14.76,0.657170,1.521677,"POLYGON ((468304.800 4983820.800, 468304.800 4..."


In [31]:
obj.describe()

,raster_val,imgb1_max,imgb1_mean,imgb2_max,imgb2_mean,imgb3_max,imgb3_mean,imgb4_max,imgb4_mean,nDSM_mean,roughness_mean,ndvi_mean,area_zs,compactness,roundness
count,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,9033.000000,5270.000000,5259.000000,5269.000000,9033.000000,9033.000000,9033.000000
mean,4517.000000,87.438282,61.142690,87.693457,63.626343,76.348279,55.630130,85.700100,51.341469,2.890653,3.735777,-0.150868,144.217762,0.559391,2.131951
std,2607.746824,80.832053,60.440066,79.430565,60.233265,71.472467,52.404200,85.796136,59.857532,3.391132,4.032175,0.371883,293.472714,0.207961,1.070550
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.021763,0.000000,-0.913682,0.360000,0.047589,1.273240
25%,2259.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063125,0.368812,-0.425017,0.360000,0.375953,1.273240
50%,4517.000000,94.000000,65.362903,104.000000,69.586057,85.000000,70.303371,72.000000,28.644444,1.495482,2.293838,-0.135668,45.720000,0.512254,1.952156
75%,6775.000000,163.000000,101.583333,159.000000,114.385519,134.000000,87.897872,175.000000,96.032653,4.760528,6.020273,0.204322,177.840000,0.785398,2.659910
max,9033.000000,255.000000,231.715447,255.000000,228.756098,255.000000,222.398374,255.000000,211.557692,18.680612,22.409891,0.420029,6719.400000,0.785398,21.013426


In [32]:
# Field names
ndvi_mean = 'ndvi_mean'
ndsm_mean = 'nDSM_mean'
roughness_mean = 'roughness_mean'
imgb1_mean = 'imgb1_mean'
imgb2_mean = 'imgb2_mean'
imgb3_mean = 'imgb3_mean'
imgb4_mean = 'imgb4_mean'
roundness = 'roundness'
# Class names
trees = 'trees'
water = 'water'
open_green = 'open_green'
buildings = 'buildings'
roads_pave = 'roads_pavement'
shadow = 'shadow'
# Add a field to hold the class
CLASS = 'class'
obj[CLASS] = None


def apply_rules(gdf, rules, class_name, unclass_only=True):
    """Function to apply rules"""
    # Look up strings operators to get function
    op_lut = {'>': operator.gt,
              '<': operator.lt}
    
    matches = copy.deepcopy(gdf)
    for field, op, val in rules:
        if isinstance(op, str):
            # if  a string is passed, get the function
            op = op_lut[op]
        matches = matches[op(matches[field], val)]
    root_logger.info(f'Objects to be classified as {class_name}: {len(matches)}')
    if unclass_only:
        # Index in matches and not classified yet
        gdf.loc[gdf.index.isin(matches.index) & gdf[CLASS].isnull(), CLASS] = class_name
    else:
        # Index in matches - would overwrite class if present
        gdf.loc[gdf.index.isin(matches.index), CLASS] = class_name
    root_logger.info(f'Objects classified as {class_name}: {len(gdf[gdf[CLASS]==class_name])}')
    root_logger.debug(f'Remaining unclassified: {len(gdf[gdf[CLASS].isnull()])}')
    return gdf

In [33]:
# Trees
tree_rules = [
    (ndvi_mean, '>', 0),
    (ndsm_mean, '>', 0.75),
    (roughness_mean, '>', 1.25)
]
# Water
water_rules = [
    (ndvi_mean, '<', 0),
    (roughness_mean, '<', 0.13),
    (ndsm_mean, '<', 1),
    (imgb4_mean, '<', 10)
]
# Open Green Space
open_rules = [
    (ndvi_mean, '>', 0.1),
    (ndsm_mean, '<', 1)
]
# Buildings
build_rules = [
    (ndvi_mean, '<', 0),
    (ndsm_mean, '>', 2),
]
# Roads and pavement
roads_rules = [
    (ndvi_mean, '<', 0),
    (ndsm_mean, '<', 1),
    (roughness_mean, '<', 1.5),
    (roundness, '>', 2)
]
# Shadow
shadow_rules = [
    (imgb1_mean, '<', 150),
    (imgb2_mean, '<', 150),
    (imgb3_mean, '<', 150),
    (imgb4_mean, '<', 150)
]
classes = [trees, water, open_green, buildings, roads_pave, shadow]
rules = [tree_rules, water_rules, open_rules, build_rules, roads_rules, shadow_rules]
classes_rules = zip(classes, rules)
for class_name, class_rules in classes_rules:
    root_logger.info(f'Classifying: {class_name}')
    obj = apply_rules(obj, class_rules, class_name)

2021-04-23 12:41:57,551 - root - INFO - Classifying: trees
2021-04-23 12:41:57,613 - root - INFO - Objects to be classified as trees: 1634
2021-04-23 12:41:57,636 - root - INFO - Objects classified as trees: 1634
2021-04-23 12:41:57,651 - root - INFO - Classifying: water
2021-04-23 12:41:57,736 - root - INFO - Objects to be classified as water: 463
2021-04-23 12:41:57,751 - root - INFO - Objects classified as water: 463
2021-04-23 12:41:57,778 - root - INFO - Classifying: open_green
2021-04-23 12:41:57,873 - root - INFO - Objects to be classified as open_green: 424
2021-04-23 12:41:57,883 - root - INFO - Objects classified as open_green: 387
2021-04-23 12:41:57,899 - root - INFO - Classifying: buildings
2021-04-23 12:41:57,952 - root - INFO - Objects to be classified as buildings: 995
2021-04-23 12:41:57,968 - root - INFO - Objects classified as buildings: 995
2021-04-23 12:41:57,999 - root - INFO - Classifying: roads_pavement
2021-04-23 12:41:58,068 - root - INFO - Objects to be class

Other rules are possible, but complicated.  
* OR rules  
* adjacency


In [34]:
# Write classification
out_class = out_vec.replace('/seg', '/classified')
# write_gdf(obj, out_class)
print(out_class)

results\seg\naip_m_4509361_se_15_060_20190727_aoi_bst100ni100s0spec0x6spat25.gpkg/classified
